# Pre/Post Analysis and Diff-in-Diff Plotting

In [1]:
%pip install -q statsmodels
%pip install -q altair
%pip install -q pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import statsmodels.formula.api as smf
import altair as alt
import numpy as np

In [3]:
def regression_model(data, year_start, year_end, color, yvar, xvar, legend, alpha=0.05):
    colour= color
    years = list(np.arange(year_start, year_end,1))

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    predictions['Treat'] = f"{legend}"
    reg = alt.Chart(predictions).mark_line().encode(x=xvar, y=alt.Y(yvar), color = alt.value(f"{colour}"), opacity=alt.Opacity("Treat", legend=alt.Legend(title="Legend")))
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format='.0f', values=years)),
            y=alt.Y("ci_low", title="Opioid Shipments per Capita in Milligrams (MME)", scale=alt.Scale(zero=False)),
            y2="ci_high",
            color=alt.value(f"{color}")
        )
    )

    chart = ci + reg
    return predictions, chart 

In [4]:
def plotting_chart(policy_year, year_start, year_end, color, data, yvar, xvar, legend, alpha=0.05):
    pl_year = policy_year
    pol_year = []
    pol_year.append(int(pl_year))
    years = list(np.arange(year_start, year_end, 1))

    # Plotting chart
    fit, reg_chart = regression_model(color=color, data= data, yvar=yvar, xvar=xvar, legend=legend, alpha=alpha, year_start=year_start, year_end=year_end)
    policy = pd.DataFrame({"Year": pol_year})
    rule = (
        alt.Chart(policy)
        .mark_rule(color="black")
        .encode(alt.X("Year:Q", title ="Year",axis=alt.Axis(values=years)))
    )
    return (reg_chart + rule).properties(width=800, height=500)

## Washington

In [5]:
washington = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/washington_PC.parquet')
oregon = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/oregon_PC.parquet')
illinois = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/illinois_PC.parquet')
ohio = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/ohio_PC.parquet')

In [6]:
treatment_state = washington[['TransactionYear', 'per_capita_MME_mg']]
ctrl_states = pd.concat([oregon,illinois,ohio], ignore_index=True)
ctrl_states = ctrl_states[['TransactionYear', 'per_capita_MME_mg']]

In [7]:
pre_years = [2009, 2010, 2011]
pre_TreatmentState = treatment_state.loc[treatment_state['TransactionYear'].isin(pre_years)]
pre_crtl = ctrl_states.loc[ctrl_states['TransactionYear'].isin(pre_years)]
post_years = [2012, 2013, 2014]
post_TreatmentState = treatment_state.loc[treatment_state['TransactionYear'].isin(post_years)]
post_crtl = ctrl_states.loc[ctrl_states['TransactionYear'].isin(post_years)]

In [8]:
pre_FL_plot = plotting_chart(2012, 2009, 2015, "#063970", pre_TreatmentState, "per_capita_MME_mg", "TransactionYear","Washington", alpha=0.05)
post_FL_plot = plotting_chart(2012, 2009, 2015, "#063970", post_TreatmentState, "per_capita_MME_mg", "TransactionYear","Washington", alpha=0.05)

In [9]:
pre_post_final = pre_FL_plot + post_FL_plot
pre_post_final.properties(title = "Pre-Post Analysis of Regulations on Opioid Shipments for Washington")

alt.LayerChart(...)

In [10]:
pre_crtl_plot = plotting_chart(2012, 2009, 2015, "#873e23", pre_crtl, "per_capita_MME_mg", "TransactionYear","Control States - OR, IL, OH", alpha=0.05)
post_crtl_plot = plotting_chart(2012, 2009, 2015, "#873e23", post_crtl, "per_capita_MME_mg", "TransactionYear","Control States - OR, IL, OH", alpha=0.05)

In [11]:
diff_in_diff_final = pre_FL_plot + post_FL_plot + pre_crtl_plot + post_crtl_plot
diff_in_diff_final.properties(title = "Diff-in-Diff Analysis of Regulations on Opioid Shipments for Washington vs Control States")

alt.LayerChart(...)

## Florida

In [12]:
florida = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/florida_PC.parquet')
ohio = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/ohio_PC.parquet')
tennessee = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/tennessee_PC.parquet')
westvirginia = pd.read_parquet('../../00_Data/Transaction_Data/06_PerCapitaMME/westvirginia_PC.parquet')

In [13]:
treatment_state = florida[['TransactionYear', 'per_capita_MME_mg']]
ctrl_states = pd.concat([ohio,tennessee,westvirginia], ignore_index=True)
ctrl_states = ctrl_states[['TransactionYear', 'per_capita_MME_mg']]

In [14]:
pre_years = [2007, 2008, 2009]
pre_TreatmentState = treatment_state.loc[treatment_state['TransactionYear'].isin(pre_years)]
pre_crtl = ctrl_states.loc[ctrl_states['TransactionYear'].isin(pre_years)]
post_years = [2010, 2011, 2012]
post_TreatmentState = treatment_state.loc[treatment_state['TransactionYear'].isin(post_years)]
post_crtl = ctrl_states.loc[ctrl_states['TransactionYear'].isin(post_years)]

In [15]:
pre_FL_plot = plotting_chart(2010, 2007, 2012, "#063970", pre_TreatmentState, "per_capita_MME_mg", "TransactionYear","Florida", alpha=0.05)
post_FL_plot = plotting_chart(2010, 2007, 2012, "#063970", post_TreatmentState, "per_capita_MME_mg", "TransactionYear","Florida", alpha=0.05)

In [16]:
pre_post_final = pre_FL_plot + post_FL_plot
pre_post_final.properties(title = "Pre-Post Analysis of Regulations on Opioid Shipments for Florida")

alt.LayerChart(...)

In [17]:
pre_crtl_plot = plotting_chart(2012, 2009, 2015, "#873e23", pre_crtl, "per_capita_MME_mg", "TransactionYear","Control States - OH, TN, WV", alpha=0.05)
post_crtl_plot = plotting_chart(2012, 2009, 2015, "#873e23", post_crtl, "per_capita_MME_mg", "TransactionYear","Control States - OH, TN, WV", alpha=0.05)

In [18]:
diff_in_diff_final = pre_FL_plot + post_FL_plot + pre_crtl_plot + post_crtl_plot
diff_in_diff_final.properties(title = "Diff-in-Diff Analysis of Regulations on Opioid Shipments for Florida vs Control States")

alt.LayerChart(...)